In [167]:
# Sabatino Paper

import numpy as np
import pandas as pd
import CoolProp.CoolProp as CP

R = 8.314 # J/molK
MW_CO2 = 44.01e-3 # kg/mol
MW_H2O = 18.015e-3 # kg/mol

# Fitted Parameters
a = 116.87
b = 15.00

In [169]:
## Parameter dictionaries:
# Toth
TOTH_PAR = {
    "Tri-PE-MCM-41": {
        "T0": 298,
        "chem": dict(ns0=2.897, b0=3.135e6, dH=117.8e3,
                     t0=0.236, alpha=0.482, chi=0.207),
        "phys": dict(ns0=8.208, b0=0.636,    dH=  2.64e3,
                     t0=0.872, alpha=0.003, chi=4.539)
    },
    "MIL-101-PEI-800": {
        "T0": 270,
        "chem": dict(ns0=3.450, b0=9.96e6,  dH= 68.3e3,
                     t0=0.243, alpha=1.802, chi=4.504),
        "phys": dict(ns0=6.205, b0=93.2,    dH= 40.1e3,
                     t0=0.163, alpha=2.287, chi=0.579)
    },
    "Lewatit-VP-OC-106": {
        "T0": 278,
        "chem": dict(ns0=2.211, b0=2.54e6,  dH= 91.2e3,
                     t0=0.442, alpha=0.520, chi=0.000),
        "phys": dict(ns0=1.840, b0=151.0,   dH=  5.19e3,
                     t0=0.636, alpha=2.407, chi=7.186)
    },
    "APDES-NFC": {   # single site
        "T0": 296,
        "chem": dict(ns0=2.310, b0=0.560e6, dH= 50.0e3,
                     t0=0.368, alpha=0.368, chi=2.501),
        "phys": None
    }
}

# GAB
GAB_PAR = {
    "MCF-APS-hi": dict(C0=21.375, dHC=-1.957e3,  K0=0.013, dHK=-1.000e3,
                          Cm0=0.0510, beta=995.3),
    "Lewatit-VP-OC-106": dict(C0=1.715, dHC=-24.636e3, K0=0.008, dHK= 9.438e3,
                              Cm0=4.697, beta=3083.0),
    "APDES-NFC-1": dict(C0=6.86, dHC=-4.120e3, K0=2.27, dHK=-2.530e3,
                        Cm0=0.0208, beta=1540),
    "APDES-NFC-2": dict(C0=6.86, dHC=-5.088e3, K0=2.27, dHK=-3.443e3,
                        Cm0=0.0208, beta=1797),
}

# Solid properties
SOLID_PAR = {
    "APDES-NFC": dict(d_p=1.3e-3, rho_s=1589.9, rho_p=61.0, f_b=0.908,
                      rho_b=55.4, cp=2070), 

    "Tri-PE-MCM-41": dict(d_p=1.0e-3, rho_s=2120, rho_p=550, f_b=0.582,
                          rho_b=320.0, cp=1000),       

    "MIL-101(Cr)-PEI-800": dict(d_p=0.996e-3, rho_s=1590, rho_p=500, f_b=None,
                                rho_b=377.1, cp=892.5), 
       
    "Lewatit VP OC 106": dict(d_p=0.688e-3, rho_s=1070, rho_p=880, f_b=0.773,
                              rho_b=680.0, cp=1580), 
 
    "Exemplary-isotherm": dict(d_p=0.996e-3, rho_s=1590, rho_p=497.8, f_b=0.754,
                               rho_b=375.4, cp=1514.2),
}

# Delta H (not sure yet here)
DELTA_H = {
    "Tri-PE-MCM-41": dict(CO2=96.7e3,  H2O=58.0e3),
    "MIL-101-PEI-800":dict(CO2=83.9e3, H2O=49.0e3),
    "Lewatit-VP-OC-106":dict(CO2=58.2e3, H2O=53.2e3),
    "APDES-NFC":     dict(CO2=58.8e3,  H2O=49.0e3)
}

In [171]:
# Toth Model
def toth_coeff(site, T, T0):
    ns = site["ns0"] * np.exp(site["chi"] * (1 - T/T0))
    b  = site["b0"] * np.exp(site["dH"]/(R*T0) * (T0/T - 1))
    t  = site["t0"] + site["alpha"] * (1 - T0/T)
    return ns, b, t

# Toth Equation:
def q_CO2_Toth(p_CO2_kPa, T, sorb):
    p_MPa = p_CO2_kPa / 1000
    par = TOTH_PAR[sorb]
    T0 = par["T0"]

    # chemisorption
    ns_c, b_c, t_c = toth_coeff(par["chem"], T, T0)
    q_chem = ns_c * b_c*p_MPa / (1 + (b_c*p_MPa)**t_c)**(1/t_c)

    # physisorption
    if par["phys"] is not None:
        ns_p, b_p, t_p = toth_coeff(par["phys"], T, T0)
        q_phys = ns_p * b_p*p_MPa / (1 + (b_p*p_MPa)**t_p)**(1/t_p)
    else:
        q_phys = 0.0

    return q_chem + q_phys

In [173]:
# GAB Model
def p0_H2O_kPa(T): # saturation pressure of water, kPa
    return CP.PropSI('P','T',T,'Q',1,'Water') / 1000.0


def q_H2O_GAB(pH2O_kPa, T, sorb):
    p = GAB_PAR[sorb]
    a_w  = pH2O_kPa / p0_H2O_kPa(T)

    C  = p["C0"]  * np.exp(p["dHC"] / (R * T))
    K  = p["K0"]  * np.exp(p["dHK"] / (R * T))
    Cm = p["Cm0"] * np.exp(p["beta"] / T)

    return (Cm * C * K * a_w) / ((1 - K*a_w) * (1 + (C - 1) * K * a))


In [175]:
# Finds equilibrium temperature to run Toth Model based on RH
def Teq_withRH(pH2O_kPa, T):
    aw   = pH2O_kPa / p0_H2O_kPa(T)
    Teq  = T - a * (278.0/T)**b * aw      # Eq. 17
    return Teq
    

In [75]:
CONTACTOR_PAR = {
    # single sorbent plate inside the cube module
    "plate": {
        "length":      0.05,       # m
        "radius_i":    0.005,      # internal radius (m)
        "radius_o":    0.005001,   # external radius (m)
        "cp_wall":   2.457e6,      # J/(Km3)
        "h_W":          6.7        # W/(m2K)  
    },
    # packed-bed “cube” module (1.5 m edge)
    "module": {
        "LxWxH": (1.5, 1.5, 1.5),  # m
        "void_fraction": 0.60      # ε = 1 − V_sorbent / V_aircontactor
    }
}

In [78]:
def W_AirBlower(dP, V_air_dot, eta_AB):
    return dP * V_air_dot / eta_AB


def W_pump_isentropic(R, m_dot, cp, T, p_in, p_out, eta_VP):
    gamma  = cp / (cp - R)
    ratio  = (p_out / p_in)**((gamma - 1.0) / gamma) - 1.0
    return m_dot * cp * T * ratio / eta_VP

In [102]:
T = 320
CO2_conc = 430 # ppm
p_CO2 = 101325/1000 * (CO2_conc/1000000) # kPa

cp_CO2 = CP.PropsSI('C','T',T,'P',p_CO2,'CO2')      # J kg⁻¹ K⁻¹
print(cp_CO2)

866.1970583237526


In [146]:
T = 280

In [148]:
P = 1400
vap = CP.PropsSI('C', 'T', T, 'P', P, 'Water')


print(vap)


4201.348790423362


In [ ]:
need to calculate P of water vapor given humidity stuff

In [ ]:
def cp_CO2(T): 
    return CP.PropsSI('C', 'T', T, 'P', 101325, 'CO2')
def cp_H2O(T):  
    return CP.PropsSI('C', 'T', T, 'P', 101325, 'Water')
def cp_H2O_vap(T):         # low-pressure vapor to stay in gas phase
    return CP.PropsSI('C', 'T', T, 'P', 1000.0, 'Water') 

def Q_total(T_ads):
    
T_des = 373.0
T_sat = 373.0

MW_CO2 = 44.01e-3 #kg/mol
MW_H2O = 18.015e-3 #kg/mol

cp_CO2 = cp_CO2(T_des)
cp_H2O = cp_H2O(T_ads)
cp_vap = cp_H2O_vap(T_des)

Q_rxn_CO2 = dH_CO2 / MW_CO2
Q_rxn_H2O = dH_H2O * dq_H2O / (dq_CO2 * MW_CO2)

Q_sens_sorb = (cp_sorbent * (T_des-T_ads) / (dq_CO2 * MW_CO2))
Q_sens_CO2  = cp_CO2 * (T_des-T_ads)
Q_sens_H2O  = (cp_H2O * (T_sat-T_ads) + cp_vap * (T_des-T_sat)) * (dq_H2O * MW_H2O) / (dq_CO2 * MW_CO2)     

Q_total =  Q_rxn_CO2 + Q_rxn_H2O + Q_sens_sorb + Q_sens_CO2 + Q_sens_H2O